# Artificial Neural Networks

---
## Part 1 - Data Preprocessing

#### Importing the Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.3.0'

#### Importing the dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


#### Encoding categorical data

In [6]:
# Label Encoding the Gender Feature
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [7]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [8]:
# One-Hot Encoding the values of the Geography feature
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), \
                                     [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [9]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


#### Splitting the dataset into training and test set

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,\
                                                   random_state = 0)

#### Feature Scaling

In [11]:
# Feature Scaling is compulsory for Deep Learning.
# All Feature Vectors (of both Training Set and Test Set) need 
#    to be feature-scaled
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

---
## Part 2 - Building the ANN

#### Initializing the ANN

In [12]:
# Create an ANN as a sequence of layers
ann = tf.keras.models.Sequential()

#### Adding the input layer and the first hidden layer

In [13]:
# Adding 1st Layer of 6 Neurons with rectifier function as activation function
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

#### Adding the second hidden layer

In [14]:
# Adding 2nd Layer of 6 Neurons with rectifier function as activation function
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

#### Adding the output layer

In [15]:
# Adding Output layer of 1 Neuron for binary output
# Sigmoid Activation function is used for outputting 0 or 1
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

---
## Part 3 - Training the ANN

#### Compiling the ANN

In [16]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', \
            metrics = ['accuracy'])

#### Training the ANN on the Training Set

In [17]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 2ms/step - loss: 0.6551 - accuracy: 0.6582
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4916 - accuracy: 0.8040
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4401 - accuracy: 0.8156
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4233 - accuracy: 0.8189
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4147 - accuracy: 0.8198
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4075 - accuracy: 0.8216
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4010 - accuracy: 0.8236
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3947 - accuracy: 0.8278
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3877 - accuracy: 0.8332
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3796 - accura

---
## Part 4 - Making the predictions and evaluating the model

#### Predicting the result of a single observation

__Homework__

Use our ANN model to predict if the customer with the following information will leave the bank:
- Geography: France
- Credit Score: 600
- Gender: Male
- Age: 40 years old
- Tenure: 3 years
- Balance: \$ 60000
- Number of Products: 2
- Does this customer have a credit card ? Yes
- Is the customer an Active Member: Yes
- Estimated Salary: \$ 50000

So, should we say goodbye to the customer ?

__Solution__

In [18]:
ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))

array([[0.0366987]], dtype=float32)

In [19]:
# Prediction
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) \
      > 0.5)

[[False]]


Therefore our ANN model predicts that the customer stays in the bank

#### Predicting the Test set results

In [20]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), \
                     y_test.reshape(len(y_test), 1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


#### Making the Confusion Matrix

In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1502   93]
 [ 174  231]]


0.8665